In [2]:
import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [3]:
import time

In [4]:
vic_data = pd.read_csv('victoria.csv', parse_dates=[0])

In [5]:
BATTERY_POWER = 300
BATTERY_CAPACITY = 580
EFFICIENCY = 0.9
MARGINAL_LOSS_FACTOR = 0.991
FIXED_OPERATIONS_MAINTENANCE = 8.1
TRADING_PRICE = 'Regions VIC Trading Price ($/MWh)'

In [6]:
CHARGING_LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.25

DISCHARGING_LOOKAHEAD = 10
DISCHARGING_PERCENTILE = 0.75

In [7]:
vic_price = vic_data[['Time (UTC+10)',TRADING_PRICE]]

In [8]:
#vic_price['charging_or_not'] = 0
#vic_price['discharging_or_not'] = 0

vic_price['raw_power'] = 0

vic_price['market_dispatch'] = 0

vic_price['opening_capacity'] = 0
vic_price['closing_capacity'] = 0

vic_price['revenue'] = 0

In [9]:
vic_price

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,market_dispatch,opening_capacity,closing_capacity,revenue
0,2018-01-01 00:00:00,90.43,0,0,0,0,0
1,2018-01-01 00:30:00,92.46,0,0,0,0,0
2,2018-01-01 01:00:00,87.62,0,0,0,0,0
3,2018-01-01 01:30:00,73.08,0,0,0,0,0
4,2018-01-01 02:00:00,70.18,0,0,0,0,0
...,...,...,...,...,...,...,...
63452,2021-08-14 22:00:00,49.93,0,0,0,0,0
63453,2021-08-14 22:30:00,62.86,0,0,0,0,0
63454,2021-08-14 23:00:00,32.26,0,0,0,0,0
63455,2021-08-14 23:30:00,25.10,0,0,0,0,0


In [10]:
CHARGE_LEVEL = 100
DISCHARGE_LEVEL = 110

In [11]:
start_time = time.time()

for i in list(vic_price.index)[1:]:
    
    
    # update opening capacity
    if(i != 0):
        vic_price.at[i, 'opening_capacity'] = vic_price.at[i-1, 'closing_capacity']
    
    
    
    # finding raw_power -- this uses the modified function Akira gave us in the last announcement
    if(vic_price.at[i,TRADING_PRICE] < CHARGE_LEVEL):
        vic_price.at[i,'raw_power'] = -1*min(BATTERY_POWER, 
                                           (BATTERY_CAPACITY-vic_price.at[i,'opening_capacity'])/EFFICIENCY*2)

    if(vic_price.at[i,TRADING_PRICE] > DISCHARGE_LEVEL):
        vic_price.at[i,'raw_power'] = min(BATTERY_POWER, vic_price.at[i,'opening_capacity']/EFFICIENCY*2)
        
        
        
    # finding market_dispatch 
    if(vic_price.at[i,'raw_power'] < 0):
        vic_price.at[i,'market_dispatch'] = vic_price.at[i,'raw_power']/2
        
    else:
        # EFFICIENCY is already in decimal (0.9) so no need to divide by 100
        vic_price.at[i,'market_dispatch'] = vic_price.at[i,'raw_power']/2 * EFFICIENCY 
        
        
        
    # finidng closing_capacity   
    if(vic_price.at[i,'market_dispatch'] < 0):
        thecondition = vic_price.at[i,'opening_capacity'] - (vic_price.at[i,'market_dispatch'] * EFFICIENCY)
    else:
        thecondition = vic_price.at[i,'opening_capacity'] - (vic_price.at[i,'market_dispatch'] * (100/(EFFICIENCY*100)))
    
    vic_price.at[i,'closing_capacity'] = round(max(0, min(thecondition, BATTERY_CAPACITY)),0)
      
        
        
    #finding revenue        
    if(vic_price.at[i,'market_dispatch'] < 0):
        vic_price.at[i,'revenue'] = round(vic_price.at[i,'market_dispatch'] * vic_price.at[i,TRADING_PRICE] * (1/MARGINAL_LOSS_FACTOR),0)
    else:
        vic_price.at[i,'revenue'] = round(vic_price.at[i,'market_dispatch'] * vic_price.at[i,TRADING_PRICE] * MARGINAL_LOSS_FACTOR,0)


end_time = time.time()
loop_time = end_time - start_time

In [12]:
vic_price.head(20)

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,market_dispatch,opening_capacity,closing_capacity,revenue
0,2018-01-01 00:00:00,90.43,0,0,0,0,0
1,2018-01-01 00:30:00,92.46,-300,-150,0,135,-13995
2,2018-01-01 01:00:00,87.62,-300,-150,135,270,-13262
3,2018-01-01 01:30:00,73.08,-300,-150,270,405,-11062
4,2018-01-01 02:00:00,70.18,-300,-150,405,540,-10623
5,2018-01-01 02:30:00,67.43,-88,-44,540,580,-2994
6,2018-01-01 03:00:00,66.31,0,0,580,580,0
7,2018-01-01 03:30:00,67.72,0,0,580,580,0
8,2018-01-01 04:00:00,65.50,0,0,580,580,0
9,2018-01-01 04:30:00,64.50,0,0,580,580,0


In [13]:
sum(vic_price.loc[(vic_price['Time (UTC+10)'] >= '2020-07-17') & (vic_price['Time (UTC+10)'] < '2020-07-18')]['revenue'])
# this method gives incorrect answer, not even close to the answer 44258

23475

In [11]:
vic_price.to_csv('vic_price_algo1.csv')

In [12]:
print("Total Revenue:", vic_price['revenue'].sum())
print("Average Revenue (per day):", vic_price['revenue'].sum()/(len(vic_price)/48))

Total Revenue: 34844960
Average Revenue (per day): 26357.345604109873


- The total revenue here is higher than the total revenue in the given file, I think this is right cuz the file is only up to 2021/07/01, this dataset has more data than the one in the given example

- The dataset used contains all data (not only training but also the validation and testing part)

- It is observed that `algorithm 3 definitely generates more revenue than algorithm 1`.